In [1]:
import pandas as pd
import json 
import numpy as np
import math
import re
from collections import Counter
from tqdm import tqdm
tqdm.pandas()

In [2]:
cd "C:\Users\nbzhy\Projects\99designs_project_brief_data_scraping\data_20220301"

C:\Users\nbzhy\Projects\99designs_project_brief_data_scraping\data_20220301


In [3]:
# load data
df1 = pd.read_csv('cleaned_contest_data_after_change_part1.csv')
df2 = pd.read_csv('cleaned_contest_data_after_change_part2.csv')
df3 = pd.read_csv('cleaned_contest_data_after_change_part3.csv')
df4 = pd.read_csv('cleaned_contest_data_after_change_part4.csv')
df_before = pd.read_csv('cleaned_contests_before_change_v3.csv')

In [4]:
df = pd.concat([df1,df2,df3,df4,df_before])


In [5]:
# get all solvers from 2010-12-14 to 2011-03-07
df_month = df[(df['start_date']>='2010-12-14') & (df['start_date']<='2011-03-07')].copy()

In [6]:
df_contests_before = df_month[df_month['start_date']<'2011-01-25'].copy()
df_contests_after = df_month[df_month['start_date']>='2011-01-25'].copy()

In [7]:
#columns_for_reference
df_contests_before.columns

Index(['url', 'seeker', 'start_time', 'name_to_incorporate',
       'slogan_to_incorporate', 'description', 'industry', 'logo_types',
       'logo_to_be_used', 'colors_to_explore', 'other_color_requirements',
       'style_attributes', 'design_inspiration_count', 'attachments_count',
       'other_notes', 'entry_count', 'deleted_entry_count', 'deleted',
       'withdrawn', 'declined', 'prize', 'entries', 'deleted_entries',
       'winners', 'tags', 'guaranteed', 'blind', 'not_blind', 'fast_tracked',
       'gold', 'platinum', 'prize_log', 'start_date', 'number_of_winners',
       'multi_winner', 'total_submission', 'number_of_participants',
       'number_of_submission_per_participants', 'overview', 'company_name',
       'description_of_organization_and_audience', 'requirements'],
      dtype='object')

In [8]:
df_solver_quality = pd.read_csv('solver_profile_from_beginning.csv')
df_solver_quality.head()

,user_id,total_contest_participation,contest_won,prize_won,winning_rate
0,524290,0,0,0,0.0
1,393218,8,0,0,0.0
2,327684,24,0,0,0.0
3,425990,31,0,0,0.0
4,425991,6,0,0,0.0


In [9]:
def winning_rate(row):
    if row['total_contest_participation'] <= 5:
        return 0
    else:
        return row['contest_won'] / row['total_contest_participation'] * 100
    

In [10]:
df_contests_before.shape

(1288, 42)

In [11]:
df_contests_after.shape

(1935, 42)

In [12]:
df_solver_quality.describe()

,user_id,total_contest_participation,contest_won,prize_won,winning_rate
count,12677.000000,12677.000000,12677.000000,12677.000000,12677.000000
mean,441464.762010,30.697878,1.147906,301.018380,0.012050
std,98179.978582,89.094980,4.788058,1265.564514,0.039731
min,22296.000000,0.000000,0.000000,0.000000,0.000000
25%,373643.000000,0.000000,0.000000,0.000000,0.000000
50%,478549.000000,1.000000,0.000000,0.000000,0.000000
75%,510167.000000,19.000000,0.000000,0.000000,0.000000
max,775124.000000,2791.000000,131.000000,32275.000000,1.000000


In [13]:
np.percentile(df_solver_quality['winning_rate'], [95, 90, 85])


array([0.07407407, 0.04255319, 0.02264537])

In [14]:
# number of contests participated 
def get_number_of_contests_participated(row):

    user_id = int(row['user_id'])
    
    contest_participation = []
    prize_total = 0

    user_id = str(user_id)
    entries = df_contests_before['entries'].values
    deleted_entries = df_contests_before['deleted_entries'].values
    prizes = df_contests_before['prize'].values

    for entry1, entry2, prize in zip(entries, deleted_entries, prizes):
        if user_id in str(entry1) or user_id in str(entry2):
            contest_participation.append(True)
            prize_total += prize

        else:
            contest_participation.append(False)

    contest_total = sum(contest_participation)

    if contest_total == 0:
        return 0, 0
    else:
        return contest_total, prize_total / contest_total


    

In [15]:
df_solver_quality[['total_contest_participation_before', 'avg_prize_before']] = df_solver_quality.progress_apply(get_number_of_contests_participated, axis=1, result_type="expand")

100%|████████████████████████████████████████████████████████████████████████████| 12677/12677 [04:46<00:00, 44.25it/s]


In [16]:
# contest won num
winner_ids = []
contest_prizes = []

for index, row in df_contests_before.iterrows():

    winners = re.findall('entry-\d{1,3}', row['winners'])
    if winners:
        for winner in winners:
            pattern = winner + "', 'participant_id': '(\d{5,7})'"
            # print(pattern)
            winners_1 = re.findall(pattern, row['entries']) 
            winners_2 = re.findall(pattern, row['deleted_entries'])
            winner_ids += winners_1
            winner_ids += winners_2
            prize = row['prize']
            num_winners = len(winners_1 + winners_2)
            if num_winners >= 1:
                for i in range(num_winners):
                    contest_prizes.append(int(prize / num_winners))
            else:
                pass


In [17]:
counter = Counter(winner_ids)
contest_won = dict(counter)

In [18]:
df_prize = pd.DataFrame({'user_id': winner_ids, 'prize': contest_prizes})
df_prize = df_prize.groupby('user_id', as_index=False).sum()
df_prize.head()

,user_id,prize
0,111301,542
1,114370,300
2,141328,400
3,147806,280
4,157293,300


In [19]:
df_solver_quality['contest_won_before'] = df_solver_quality['user_id'].apply(lambda x: contest_won[str(x)] if str(x) in contest_won else 0)
df_solver_quality['prize_won_before'] = df_solver_quality['user_id'].apply(lambda x: df_prize[df_prize['user_id'] == str(x)]['prize'].values[0] if str(x) in df_prize['user_id'].values else 0)
df_solver_quality['winning_rate_before'] = df_solver_quality['contest_won_before'] / df_solver_quality['total_contest_participation']
df_solver_quality['winning_rate_before'] = df_solver_quality['winning_rate_before'].apply(lambda x: min(x, 1))

In [20]:
# number of contests participated 
def get_number_of_contests_participated(row):

    user_id = int(row['user_id'])
    contest_participation = []
    prize_total = 0

    user_id = str(user_id)

    entries = df_contests_after['entries'].values
    deleted_entries = df_contests_after['deleted_entries'].values
    prizes = df_contests_after['prize'].values

    for entry1, entry2, prize in zip(entries, deleted_entries, prizes):
        if user_id in entry1 or user_id in entry2:
            contest_participation.append(True)
            prize_total += prize

        else:
            contest_participation.append(False)

    contest_total = sum(contest_participation)

    if contest_total == 0:
        return 0, 0
    else:
        return contest_total, prize_total / contest_total


    

In [21]:
df_solver_quality[['total_contest_participation_after', 'avg_prize_after']] = df_solver_quality.progress_apply(get_number_of_contests_participated, axis=1, result_type="expand")

100%|████████████████████████████████████████████████████████████████████████████| 12677/12677 [06:02<00:00, 35.00it/s]


In [22]:
# contest won num
winner_ids = []
contest_prizes = []

for index, row in df_contests_after.iterrows():

    winners = re.findall('entry-\d{1,3}', row['winners'])
    if winners:
        for winner in winners:
            pattern = winner + "', 'participant_id': '(\d{5,7})'"
            # print(pattern)
            winners_1 = re.findall(pattern, row['entries']) 
            winners_2 = re.findall(pattern, row['deleted_entries'])
            winner_ids += winners_1
            winner_ids += winners_2
            prize = row['prize']
            num_winners = len(winners_1 + winners_2)
            if num_winners >= 1:
                for i in range(num_winners):
                    contest_prizes.append(int(prize / num_winners))
            else:
                pass


In [23]:
counter = Counter(winner_ids)
contest_won = dict(counter)

In [24]:
df_prize = pd.DataFrame({'user_id': winner_ids, 'prize': contest_prizes})
df_prize = df_prize.groupby('user_id', as_index=False).sum()
df_prize.head()

,user_id,prize
0,126166,542
1,145891,200
2,149090,200
3,157293,400
4,160960,1200


In [25]:
df_solver_quality['contest_won_after'] = df_solver_quality['user_id'].apply(lambda x: contest_won[str(x)] if str(x) in contest_won else 0)
df_solver_quality['prize_won_after'] = df_solver_quality['user_id'].apply(lambda x: df_prize[df_prize['user_id'] == str(x)]['prize'].values[0] if str(x) in df_prize['user_id'].values else 0)
df_solver_quality['winning_rate_after'] = df_solver_quality['contest_won_after'] / df_solver_quality['total_contest_participation']
df_solver_quality['winning_rate_after'] = df_solver_quality['winning_rate_after'].apply(lambda x: min(x, 1))

In [26]:
df_solver_quality.fillna(0, inplace=True)

## solver participate in any one contest

In [27]:
df_solver_quality.describe().T

,count,mean,std,min,25%,50%,75%,max
user_id,12677.0,441464.762010,98179.978582,22296.0,373643.0,478549.000,510167.000,775124.0
total_contest_participation,12677.0,30.697878,89.094980,0.0,0.0,1.000,19.000,2791.0
contest_won,12677.0,1.147906,4.788058,0.0,0.0,0.000,0.000,131.0
prize_won,12677.0,301.018380,1265.564514,0.0,0.0,0.000,0.000,32275.0
winning_rate,12677.0,0.012050,0.039731,0.0,0.0,0.000,0.000,1.0
total_contest_participation_before,12677.0,3.546817,8.191054,0.0,0.0,1.000,3.000,138.0
avg_prize_before,12677.0,177.352360,167.164470,0.0,0.0,200.000,300.000,700.0
contest_won_before,12677.0,0.099235,0.491098,0.0,0.0,0.000,0.000,12.0
prize_won_before,12677.0,24.131419,118.193424,0.0,0.0,0.000,0.000,2890.0
winning_rate_before,12677.0,0.010242,0.089044,0.0,0.0,0.000,0.000,1.0


In [28]:
from scipy import stats

In [29]:
stats.ttest_ind(df_solver_quality['total_contest_participation_before'],df_solver_quality['total_contest_participation_after'])

Ttest_indResult(statistic=-3.032242347971787, pvalue=0.002429894489277228)

In [30]:
df_solver_quality_top = df_solver_quality[df_solver_quality['winning_rate']>=0.04]
print(stats.ttest_ind(df_solver_quality_top['total_contest_participation_before'],df_solver_quality_top['total_contest_participation_after']))
df_solver_quality_top.describe().T

Ttest_indResult(statistic=2.6533535396926573, pvalue=0.00801626978868014)


,count,mean,std,min,25%,50%,75%,max
user_id,1353.0,350563.609017,83170.635882,47956.00,288120.000000,345830.000000,417264.000000,510937.0
total_contest_participation,1353.0,105.705839,146.819423,1.00,20.000000,49.000000,132.000000,1793.0
contest_won,1353.0,7.991870,11.805904,1.00,1.000000,4.000000,9.000000,131.0
prize_won,1353.0,2074.642276,3120.800815,150.00,397.000000,946.000000,2384.000000,32275.0
winning_rate,1353.0,0.094373,0.082192,0.04,0.052632,0.071429,0.105263,1.0
total_contest_participation_before,1353.0,7.260163,10.866199,0.00,1.000000,4.000000,9.000000,131.0
avg_prize_before,1353.0,228.222658,129.063495,0.00,200.000000,244.000000,300.000000,700.0
contest_won_before,1353.0,0.487805,1.133164,0.00,0.000000,0.000000,1.000000,12.0
prize_won_before,1353.0,117.499630,267.993735,0.00,0.000000,0.000000,200.000000,2890.0
winning_rate_before,1353.0,0.011246,0.054776,0.00,0.000000,0.000000,0.001838,1.0


In [31]:
df_solver_quality_low = df_solver_quality[df_solver_quality['winning_rate']<0.04]
print(stats.ttest_ind(df_solver_quality_low['total_contest_participation_before'],df_solver_quality_low['total_contest_participation_after']))
df_solver_quality_low.describe().T

Ttest_indResult(statistic=-4.646892854883457, pvalue=3.3886874613244606e-06)


,count,mean,std,min,25%,50%,75%,max
user_id,11324.0,452325.699841,94126.720553,22296.0,393353.75,490482.5,512306.500000,775124.000000
total_contest_participation,11324.0,21.735871,74.565257,0.0,0.00,0.0,10.000000,2791.000000
contest_won,11324.0,0.330184,1.660393,0.0,0.00,0.0,0.000000,54.000000
prize_won,11324.0,89.104468,457.519179,0.0,0.00,0.0,0.000000,12856.000000
winning_rate,11324.0,0.002214,0.007346,0.0,0.00,0.0,0.000000,0.039706
total_contest_participation_before,11324.0,3.103144,7.692041,0.0,0.00,1.0,2.000000,138.000000
avg_prize_before,11324.0,171.274339,170.140868,0.0,0.00,200.0,300.000000,700.000000
contest_won_before,11324.0,0.052808,0.310609,0.0,0.00,0.0,0.000000,9.000000
prize_won_before,11324.0,12.975715,76.793238,0.0,0.00,0.0,0.000000,1996.000000
winning_rate_before,11324.0,0.010122,0.092293,0.0,0.00,0.0,0.000000,1.000000


In [32]:
df_solver_quality_top = df_solver_quality[df_solver_quality['winning_rate']>=0.07]
print(stats.ttest_ind(df_solver_quality_top['total_contest_participation_before'],df_solver_quality_top['total_contest_participation_after']))
df_solver_quality_top.describe().T

Ttest_indResult(statistic=1.664838403512776, pvalue=0.09617199729241244)


,count,mean,std,min,25%,50%,75%,max
user_id,691.0,358502.659913,84897.281798,47956.00,295653.500000,356668.000000,426392.500000,510937.0
total_contest_participation,691.0,89.494935,145.629601,1.00,11.500000,34.000000,105.500000,1793.0
contest_won,691.0,9.340087,14.313523,1.00,1.000000,4.000000,11.000000,131.0
prize_won,691.0,2401.839363,3736.572156,150.00,346.000000,973.000000,2675.000000,32275.0
winning_rate,691.0,0.133909,0.099822,0.07,0.083333,0.104478,0.142857,1.0
total_contest_participation_before,691.0,6.425470,9.619644,0.00,1.000000,3.000000,8.000000,68.0
avg_prize_before,691.0,217.883815,129.427660,0.00,200.000000,239.625000,298.000000,552.0
contest_won_before,691.0,0.539797,1.168212,0.00,0.000000,0.000000,1.000000,12.0
prize_won_before,691.0,130.056440,280.102685,0.00,0.000000,0.000000,200.000000,2890.0
winning_rate_before,691.0,0.017105,0.074499,0.00,0.000000,0.000000,0.003681,1.0


In [33]:
df_solver_quality_low = df_solver_quality[df_solver_quality['winning_rate']<0.07]
print(stats.ttest_ind(df_solver_quality_low['total_contest_participation_before'],df_solver_quality_low['total_contest_participation_after']))
df_solver_quality_low.describe().T

Ttest_indResult(statistic=-3.64837520705003, pvalue=0.0002644636723320184)


,count,mean,std,min,25%,50%,75%,max
user_id,11986.0,446247.576339,96749.217063,22296.0,382906.5,484381.5,511251.75,775124.000000
total_contest_participation,11986.0,27.308193,83.449284,0.0,0.0,1.0,16.00,2791.000000
contest_won,11986.0,0.675622,2.891312,0.0,0.0,0.0,0.00,78.000000
prize_won,11986.0,179.904806,788.067571,0.0,0.0,0.0,0.00,27148.000000
winning_rate,11986.0,0.005025,0.013797,0.0,0.0,0.0,0.00,0.069853
total_contest_participation_before,11986.0,3.380861,8.070240,0.0,0.0,1.0,3.00,138.000000
avg_prize_before,11986.0,175.015698,168.791249,0.0,0.0,200.0,300.00,700.000000
contest_won_before,11986.0,0.073836,0.405804,0.0,0.0,0.0,0.00,10.000000
prize_won_before,11986.0,18.024779,97.846645,0.0,0.0,0.0,0.00,2045.000000
winning_rate_before,11986.0,0.009846,0.089798,0.0,0.0,0.0,0.00,1.000000


## solver participate in at least one contest before

In [34]:
df_solver_quality_at_least_1_before = df_solver_quality[df_solver_quality['total_contest_participation_before']!=0]

In [35]:
df_solver_quality_at_least_1_before.describe().T

,count,mean,std,min,25%,50%,75%,max
user_id,7396.0,419326.142239,88814.510212,30858.0,352183.25,450059.5,498153.500000,565065.000000
total_contest_participation,7396.0,45.720119,108.584813,0.0,0.00,6.0,39.000000,2791.000000
contest_won,7396.0,1.733640,5.805936,0.0,0.00,0.0,1.000000,98.000000
prize_won,7396.0,453.992699,1540.516221,0.0,0.00,0.0,200.000000,30925.000000
winning_rate,7396.0,0.016254,0.040558,0.0,0.00,0.0,0.013333,0.666667
total_contest_participation_before,7396.0,6.079367,9.980403,1.0,1.00,2.0,7.000000,138.000000
avg_prize_before,7396.0,303.988084,96.946667,175.0,225.00,290.0,357.687500,700.000000
contest_won_before,7396.0,0.170092,0.633526,0.0,0.00,0.0,0.000000,12.000000
prize_won_before,7396.0,41.362088,152.424016,0.0,0.00,0.0,0.000000,2890.000000
winning_rate_before,7396.0,0.017555,0.116029,0.0,0.00,0.0,0.000000,1.000000


In [36]:
stats.ttest_ind(df_solver_quality_at_least_1_before['total_contest_participation_before'],df_solver_quality_at_least_1_before['total_contest_participation_after'])

Ttest_indResult(statistic=9.786534455067024, pvalue=1.5058608465332743e-22)

In [37]:
df_solver_quality_at_least_1_before_top = df_solver_quality_at_least_1_before[df_solver_quality_at_least_1_before['winning_rate']>=0.04]
print(stats.ttest_ind(df_solver_quality_at_least_1_before_top['total_contest_participation_before'],df_solver_quality_at_least_1_before_top['total_contest_participation_after']))
df_solver_quality_at_least_1_before_top.describe().T

Ttest_indResult(statistic=4.465827778575027, pvalue=8.381416889162538e-06)


,count,mean,std,min,25%,50%,75%,max
user_id,1096.0,354346.733577,84564.901516,47956.000000,289647.250000,351980.500000,422198.750000,510937.000000
total_contest_participation,1096.0,114.684307,147.891293,2.000000,21.000000,57.500000,146.000000,1160.000000
contest_won,1096.0,8.635949,12.055872,1.000000,2.000000,4.000000,11.000000,98.000000
prize_won,1096.0,2239.885949,3206.805740,150.000000,422.250000,1086.000000,2653.500000,30925.000000
winning_rate,1096.0,0.090321,0.064905,0.040000,0.052632,0.069767,0.101140,0.666667
total_contest_participation_before,1096.0,8.962591,11.424298,1.000000,2.000000,5.000000,11.000000,131.000000
avg_prize_before,1096.0,281.738373,73.998160,191.666667,227.142857,261.000000,318.422078,700.000000
contest_won_before,1096.0,0.602190,1.231458,0.000000,0.000000,0.000000,1.000000,12.000000
prize_won_before,1096.0,145.052007,290.993126,0.000000,0.000000,0.000000,200.000000,2890.000000
winning_rate_before,1096.0,0.013883,0.060564,0.000000,0.000000,0.000000,0.005626,1.000000


In [38]:
df_solver_quality_at_least_1_before_low = df_solver_quality_at_least_1_before[df_solver_quality_at_least_1_before['winning_rate']<0.04]
print(stats.ttest_ind(df_solver_quality_at_least_1_before_low['total_contest_participation_before'],df_solver_quality_at_least_1_before_low['total_contest_participation_after']))
df_solver_quality_at_least_1_before_low.describe().T

Ttest_indResult(statistic=8.777887608175002, pvalue=1.8788016226530423e-18)


,count,mean,std,min,25%,50%,75%,max
user_id,6300.0,430630.496508,84586.501086,30858.0,374407.75,466281.5,500248.500000,565065.000000
total_contest_participation,6300.0,33.722540,95.228945,0.0,0.00,3.0,23.000000,2791.000000
contest_won,6300.0,0.532857,2.139232,0.0,0.00,0.0,0.000000,54.000000
prize_won,6300.0,143.303968,589.029557,0.0,0.00,0.0,0.000000,12856.000000
winning_rate,6300.0,0.003369,0.008850,0.0,0.00,0.0,0.000000,0.039706
total_contest_participation_before,6300.0,5.577778,9.620476,1.0,1.00,2.0,6.000000,138.000000
avg_prize_before,6300.0,307.858827,99.904841,175.0,225.00,300.0,367.839286,700.000000
contest_won_before,6300.0,0.094921,0.411618,0.0,0.00,0.0,0.000000,9.000000
prize_won_before,6300.0,23.323333,101.780907,0.0,0.00,0.0,0.000000,1996.000000
winning_rate_before,6300.0,0.018194,0.123146,0.0,0.00,0.0,0.000000,1.000000


## solver participate in at least one contest before and at least one after

In [50]:
df_solver_quality_at_least_1_before_1_after = df_solver_quality[(df_solver_quality['total_contest_participation_before']!=0) & 
                (df_solver_quality['total_contest_participation_after']!=0)]

In [51]:
df_solver_quality_at_least_1_before_1_after.describe().T

,count,mean,std,min,25%,50%,75%,max
user_id,3640.0,409570.215110,88564.822640,34123.0,340504.000000,429135.500000,491696.500000,532033.000000
total_contest_participation,3640.0,74.935440,140.687274,0.0,1.000000,21.000000,88.000000,2791.000000
contest_won,3640.0,2.956868,7.700248,0.0,0.000000,0.000000,2.000000,98.000000
prize_won,3640.0,776.198901,2035.346327,0.0,0.000000,0.000000,620.000000,30925.000000
winning_rate,3640.0,0.022450,0.043548,0.0,0.000000,0.000000,0.031966,0.666667
total_contest_participation_before,3640.0,9.472802,12.725043,1.0,2.000000,5.000000,12.000000,138.000000
avg_prize_before,3640.0,297.540734,83.681552,175.0,233.750000,280.583333,342.000000,694.000000
contest_won_before,3640.0,0.319780,0.861017,0.0,0.000000,0.000000,0.000000,12.000000
prize_won_before,3640.0,77.519505,206.319692,0.0,0.000000,0.000000,0.000000,2890.000000
winning_rate_before,3640.0,0.028120,0.144782,0.0,0.000000,0.000000,0.000000,1.000000


In [44]:
stats.ttest_ind(df_solver_quality_at_least_1_before_1_after['total_contest_participation_before'],df_solver_quality_at_least_1_before_1_after['total_contest_participation_after'])

Ttest_indResult(statistic=1.198632619407824, pvalue=0.23070985557767007)

In [52]:
df_solver_quality_at_least_1_before_1_after_top = df_solver_quality_at_least_1_before_1_after[df_solver_quality_at_least_1_before_1_after['winning_rate']>=0.04]
print(stats.ttest_ind(df_solver_quality_at_least_1_before_1_after_top['total_contest_participation_before'],df_solver_quality_at_least_1_before_1_after_top['total_contest_participation_after']))
df_solver_quality_at_least_1_before_1_after_top.describe().T

Ttest_indResult(statistic=2.4141282248513614, pvalue=0.015891368407024473)


,count,mean,std,min,25%,50%,75%,max
user_id,755.0,355085.074172,83520.895010,47956.000000,291609.500000,354506.000000,420551.500000,510937.000000
total_contest_participation,755.0,138.707285,163.532597,2.000000,32.000000,81.000000,182.500000,1160.000000
contest_won,755.0,10.467550,13.462885,1.000000,2.000000,5.000000,13.000000,98.000000
prize_won,755.0,2711.389404,3561.374917,150.000000,571.000000,1515.000000,3350.000000,30925.000000
winning_rate,755.0,0.086503,0.059296,0.040000,0.052231,0.068966,0.095738,0.666667
total_contest_participation_before,755.0,11.456954,12.748191,1.000000,4.000000,8.000000,14.500000,131.000000
avg_prize_before,755.0,280.090964,66.140542,191.666667,232.803030,262.111111,312.906667,552.000000
contest_won_before,755.0,0.830464,1.409658,0.000000,0.000000,0.000000,1.000000,12.000000
prize_won_before,755.0,200.074172,332.147098,0.000000,0.000000,0.000000,300.000000,2890.000000
winning_rate_before,755.0,0.017230,0.067648,0.000000,0.000000,0.000000,0.011429,1.000000


In [53]:
df_solver_quality_at_least_1_before_1_after_low = df_solver_quality_at_least_1_before_1_after[df_solver_quality_at_least_1_before_1_after['winning_rate']<0.04]
print(stats.ttest_ind(df_solver_quality_at_least_1_before_1_after_low['total_contest_participation_before'],df_solver_quality_at_least_1_before_1_after_low['total_contest_participation_after']))
df_solver_quality_at_least_1_before_1_after_low.describe().T

Ttest_indResult(statistic=0.13713325483507607, pvalue=0.8909302534635736)


,count,mean,std,min,25%,50%,75%,max
user_id,2885.0,423828.891508,84219.004423,34123.0,365405.000000,450365.000,497733.000000,532033.000000
total_contest_participation,2885.0,58.246447,128.994073,0.0,0.000000,11.000,60.000000,2791.000000
contest_won,2885.0,0.991334,2.966104,0.0,0.000000,0.000,1.000000,54.000000
prize_won,2885.0,269.762565,821.095322,0.0,0.000000,0.000,150.000000,12856.000000
winning_rate,2885.0,0.005687,0.010888,0.0,0.000000,0.000,0.004566,0.039706
total_contest_participation_before,2885.0,8.953553,12.669978,1.0,2.000000,4.000,11.000000,138.000000
avg_prize_before,2885.0,302.107312,87.129414,175.0,235.272727,288.875,350.000000,694.000000
contest_won_before,2885.0,0.186135,0.574253,0.0,0.000000,0.000,0.000000,9.000000
prize_won_before,2885.0,45.447140,141.095222,0.0,0.000000,0.000,0.000000,1996.000000
winning_rate_before,2885.0,0.030971,0.158789,0.0,0.000000,0.000,0.000000,1.000000


total unique solvers 12677; unique solvers before change 7396 (new solvers 2274); unique solvers after change 8921 (new solvers 3564)

In [40]:
df_solver_quality[(df_solver_quality['total_contest_participation_before']!=0) & 
                (df_solver_quality['total_contest_participation_after']!=0)].shape

(3640, 15)

In [41]:
df_solver_quality[(df_solver_quality['total_contest_participation']==0) & 
                  (df_solver_quality['total_contest_participation_before']!=0)].shape

(2274, 15)

In [42]:
df_solver_quality[(df_solver_quality['total_contest_participation']==0) & 
                  (df_solver_quality['total_contest_participation_before']==0) & 
                 (df_solver_quality['total_contest_participation_after']!=0)].shape

(3564, 15)

In [43]:
df_solver_quality[(df_solver_quality['total_contest_participation_before']!=0) & 
                 (df_solver_quality['total_contest_participation_after']==0)].shape

(3756, 15)

In [44]:
df_change['log_diff'] = df_change['log_avg_prize_after'] - df_change['log_avg_prize_before'] 

NameError: name 'df_change' is not defined

In [ ]:
df_change_top = df_change[df_change['top']==1]
df_change_entry = df_change[df_change['top']==0]

In [ ]:
df_change_describe = df_change_top.describe().round(2).T[['count','mean','std','min','50%','max']].rename(columns={'count':'Count','mean':'Mean','std':'SD','min':'Min','50%':'Median','max':'Max'})
df_change_describe

In [ ]:
df_change_describe = df_change_entry.describe().round(2).T[['count','mean','std','min','50%','max']].rename(columns={'count':'Count','mean':'Mean','std':'SD','min':'Min','50%':'Median','max':'Max'})
df_change_describe

In [ ]:
df_change_describe = df_change.describe().round(2).T[['count','mean','std','min','50%','max']].rename(columns={'count':'Count','mean':'Mean','std':'SD','min':'Min','50%':'Median','max':'Max'})

In [ ]:
df_change_describe

In [ ]:
# df_change_describe.to_excel('solvers_change_describe.xlsx')

In [ ]:
from scipy import stats

In [ ]:
t_result_log = stats.ttest_rel(df_change['log_avg_prize_after'],df_change['log_avg_prize_before'])
t_result_log

In [ ]:
stats.ttest_rel(df_change['avg_prize_after'],df_change['avg_prize_before'])

In [ ]:
stats.ttest_rel(df_change['count_after'],df_change['count_before'])

In [ ]:
# df_change.to_excel('solver_change.xlsx')

In [ ]:
stats.ttest_1samp(df_change['diff'], 0.0)

In [ ]:
type(t_result_log)